In [25]:
n = 30

weights = {
    1: 2,   2: 5,   3: 3,   4: 23,  5: 17,
    6: 8,   7: 12,  8: 1,   9: 4,  10: 5,
    11: 7,  12: 2,  13: 4,  14: 7,  15: 20,
    16: 5,  17: 22, 18: 7,  19: 4,  20: 1,
    21: 14, 22: 0,  23: 26, 24: 16, 25: 6,
    26: 15, 27: 8,  28: 4,  29: 8,  30: 4
}

durations = {
    1: 18,  2: 71,  3: 1,   4: 4,   5: 30,
    6: 97,  7: 73,  8: 83,  9: 34, 10: 22,
    11: 33, 12: 30, 13: 35, 14: 63, 15: 25,
    16: 84, 17: 87, 18: 83, 19: 46, 20: 63,
    21: 76, 22: 60, 23: 8,  24: 85, 25: 85,
    26: 26, 27: 83, 28: 86, 29: 36, 30: 82
}


ready_dates = {
    1: 4,     2: 28,    3: 85,    4: 69,    5: 373,
    6: 397,   7: 413,   8: 425,   9: 410,   10: 478,
    11: 696,  12: 768,  13: 812,  14: 902,  15: 903,
    16: 906,  17: 1012, 18: 1113, 19: 1163, 20: 1079,
    21: 1335, 22: 1294, 23: 1330, 24: 1497, 25: 1432,
    26: 1587, 27: 1338, 28: 1341, 29: 929,  30: 982
}


due_dates = {
    1: 83,    2: 619,   3: 649,   4: 724,   5: 1117,
    6: 565,   7: 1237,  8: 593,   9: 1001, 10: 677,
    11: 1370, 12: 1167, 13: 1348, 14: 1175, 15: 1000,
    16: 1212, 17: 1238, 18: 1990, 19: 1908, 20: 1323,
    21: 1915, 22: 2143, 23: 2037, 24: 2135, 25: 1981,
    26: 1870, 27: 2029, 28: 1641, 29: 1437, 30: 1084
}

deadlines = {
    1: 2201,  2: 1270,  3: 1213,  4: 2356,  5: 2392,
    6: 636,   7: 1773,  8: 1362,  9: 3116, 10: 2786,
    11: 2037, 12: 2393, 13: 3582, 14: 1627, 15: 2062,
    16: 3511, 17: 3599, 18: 4220, 19: 2048, 20: 2703,
    21: 2856, 22: 3939, 23: 3812, 24: 3881, 25: 3363,
    26: 3804, 27: 2312, 28: 3626, 29: 3426, 30: 2384
}


successors = {
    1: [2],
    2: [8, 7, 3],
    3: [11, 6],
    4: [8, 6],
    5: [6],
    6: [19, 12, 9],
    7: [19, 12, 9],
    8: [19, 9],
    9: [23, 17, 15, 14, 13],
    10: [22, 19, 17, 14, 13],
    11: [23, 19, 17, 13],
    12: [23, 16, 13],
    13: [21, 20, 18],
    14: [21, 20, 18],
    15: [22, 20, 18],
    16: [22, 21, 18],
    17: [21, 18],
    18: [28, 25, 24],
    19: [24, 21],
    20: [25, 24],
    21: [27, 25],
    22: [30, 24],
    23: [30, 24],
    24: [27, 26],
    25: [30, 26],
    26: [29],
    27: [29],
    28: [29],
    29: [],
    30: []
}



In [26]:
predecessors = {i: [] for i in range(1, n+1)}
for i in range(1, n+1):
    for s in successors[i]:
        predecessors[s].append(i)


In [27]:
from collections import deque

indeg = {i: len(predecessors[i]) for i in range(1, n+1)}
q = deque([i for i in range(1, n+1) if indeg[i] == 0])
topo = []

while q:
    u = q.popleft()
    topo.append(u)
    for v in successors[u]:
        indeg[v] -= 1
        if indeg[v] == 0:
            q.append(v)

if len(topo) != n:
    raise ValueError("Precedence graph has a cycle!")


In [28]:
new_ready_dates = {i: ready_dates[i] for i in range(1, n+1)}
for i in topo:
    if predecessors[i]:
        new_ready_dates[i] = max(ready_dates[i], max(new_ready_dates[j] + durations[j] for j in predecessors[i]))

In [29]:
new_deadlines = {i: deadlines[i] for i in range(1, n+1)}

for i in reversed(topo):
    if successors[i]:
        new_deadlines[i] = min(new_deadlines[i], min(new_deadlines[s] - durations[s] for s in successors[i]))

In [30]:
from pysat.formula import WCNF, IDPool, CNF
from pysat.pb import PBEnc, EncType

# Variables and time horizon
# ----------------------------
t_max = max(new_deadlines.values())
t_min = min(new_ready_dates.values())
jobs = list(range(1, n+1))

vpool = IDPool()
S = {}      # S[(i,t)] = var
A = {}      # A[(i,t)] = var
valid = {}  # valid start times per job

wcnf = CNF()

print("\n=== VARIABLE GENERATION ===")

for i in jobs:
    r_i = new_ready_dates[i]
    p_i = durations[i]
    dl_i = new_deadlines[i]

    last_start = dl_i - p_i
    if last_start < r_i:
        print(f"Job {i} is impossible!")

    valid[i] = list(range(r_i, last_start + 1))

    for t in valid[i]:
        S[(i, t)] = vpool.id(("S", i, t))

    for t in range(r_i, dl_i):
        A[(i, t)] = vpool.id(("A", i, t))

print(f"S variables: {len(S)}")
print(f"A variables: {len(A)}")


=== VARIABLE GENERATION ===
S variables: 27061
A variables: 28640


In [31]:
# 1. Start Exactly Once
print("\n=== START EXACTLY ONCE ===")
start_once_cls = 0

for i in jobs:
    lits = [S[(i, t)] for t in valid[i]]

    enc = PBEnc.equals(
        lits=lits,
        bound=1,
        encoding=EncType.best
    )
    wcnf.extend(enc.clauses)
    start_once_cls += len(enc.clauses)
print(f"Start Exactly Once Clauses: {start_once_cls}" )


=== START EXACTLY ONCE ===
Start Exactly Once Clauses: 59254


In [32]:
# 2. Link S -> A (activation constraints)
print("\n=== ACTIVATION CONSTRAINTS ===")
s_to_a_cls = 0

for i in jobs:
    p_i = durations[i]
    for t_start in valid[i]:
        lit_s = S[(i, t_start)]
        for t in range(t_start, t_start + p_i):
            if t < t_max:
                lit_a = A[(i, t)]
                wcnf.append([-lit_s, lit_a])
                s_to_a_cls += 1
print(f"Link S -> A Clauses: {s_to_a_cls}")


=== ACTIVATION CONSTRAINTS ===
Link S -> A Clauses: 1440666


In [33]:
# 3. Capacity: At most one job running at time t
print("\n=== MACHINE CAPACITY ===")
machine_cap_cls = 0

for t in range(t_min, t_max):
    active = [A[(i, t)] for i in jobs if (i, t) in A]

    if len(active) > 1:
        enc = PBEnc.atmost(
            lits=active,
            bound=1,
            encoding=EncType.best
        )
        wcnf.extend(enc.clauses)
        machine_cap_cls += len(enc.clauses)
print(f"Machine Capacity Clauses: {machine_cap_cls}")


=== MACHINE CAPACITY ===
Machine Capacity Clauses: 65609


In [ ]:
# 4. Precedence constraints 
print("\n=== PRECEDENCE CONSTRAINTS ===")
prec_cls = 0

for j in jobs:
    for i in predecessors.get(j, []):    # i → j
        p_i = durations[i]

        for t_j in valid[j]:
            for t_i in valid[i]:
                if t_i + p_i > t_j:      # forbidden pair
                    wcnf.append([-S[(i, t_i)], -S[(j, t_j)]])
                    prec_cls += 1
print(f"Precedence Constraint Clauses: {prec_cls}")


=== PRECEDENCE CONSTRAINTS ===
Precedence Constraint Clauses: 17947029


In [35]:
# 4. Precedence constraints (staircase)
print("\n=== PRECEDENCE CONSTRAINTS ===")
prec_cls = 0

for i in successors:
    for j in successors[i]:      # i precedes j
        p_i = durations[i]

        for t_i in valid[i]:
            finish_i = t_i + p_i

            lits = [S[(i, t_i)]]     # include S[i,t_i]

            # include all invalid early start times for j
            for t_j in valid[j]:
                if t_j < finish_i:
                    lits.append(S[(j, t_j)])

            # if no forbidden t_j, nothing to encode
            if len(lits) <= 1:
                continue

            # Create at-most-1 constraint
            enc = PBEnc.atmost(
                lits=lits,
                bound=1,
                encoding=EncType.best
            )
            wcnf.extend(enc.clauses)
            prec_cls += len(enc.clauses)
print(f"Precedence Constraint Clauses: {prec_cls}")


=== PRECEDENCE CONSTRAINTS ===
Precedence Constraint Clauses: 40740761


In [ ]:
from pysat.examples.rc2 import RC2
from pysat.solvers import Glucose42

# SOLVE
print("\n=== SOLVING (HARD CONSTRAINTS ONLY) ===")

solver = Glucose42(wcnf)
model = solver.compute()

if model is None:
    print("UNSAT — no feasible schedule.")

else:
    print("\nFeasible schedule found.")


=== SOLVING (HARD CONSTRAINTS ONLY) ===


AttributeError: 'CNF' object has no attribute 'hard'